<a href="https://colab.research.google.com/github/Aishaamalik/Insurance-Claim-Fraud-Detection/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Insurance Claim Fraud Detection**

## **Loading data and  inspecting it**

In [1]:
!pip install openpyxl


In [2]:
import pandas as pd


In [3]:
from google.colab import files
uploaded = files.upload()


Saving Worksheet in Case Study question 2.xlsx to Worksheet in Case Study question 2.xlsx


In [4]:
excel_path = "Worksheet in Case Study question 2.xlsx"
excel_data = pd.ExcelFile(excel_path)

print("Sheet names:", excel_data.sheet_names)

df = excel_data.parse('Fraud_Detection_decsion tree')

print("Shape of the dataset:", df.shape)

df.head()


Sheet names: ['Fraud_Detection_decsion tree']
Shape of the dataset: (1000, 39)


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


# 1: Data Cleaning & Preprocessing

## 1: Import Required Libraries

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler


- pandas and numpy for data manipulation

- LabelEncoder for binary categorical encoding

- StandardScaler for feature scaling



## 2: Load the Excel File

In [6]:
from google.colab import files
uploaded = files.upload()


Saving Worksheet in Case Study question 2.xlsx to Worksheet in Case Study question 2 (1).xlsx


Loading the data:

In [7]:
excel_data = pd.ExcelFile("Worksheet in Case Study question 2.xlsx")
df = excel_data.parse('Fraud_Detection_decsion tree')


## 3: Copy the Original DataFrame

In [8]:
df_clean = df.copy()


 keeping my original data safe and useing a working copy df_clean for cleaning and preprocessing.

In [9]:
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


## 4: Handle Missing Values

### a: Replace ? with NaN



In [10]:
df_clean.replace('?', pd.NA, inplace=True)


Some columns may contain "?" as placeholders for missing values — I replace them with actual NaN.

### b: Drop Columns with Too Many Missing Values

In [11]:
missing_percentage = df_clean.isna().mean()
cols_to_drop = missing_percentage[missing_percentage > 0.5].index
df_clean.drop(columns=cols_to_drop, inplace=True)


I droped columns with more than 50% missing data (adjustable threshold).

## 5: Convert Target Variable (fraud_reported)

In [12]:
df_clean['fraud_reported'] = df_clean['fraud_reported'].map({'Y': 1, 'N': 0})


I converted the target variable into numerical form:

- Y → 1

- N → 0

## 6: Convert Date Columns

In [13]:
if 'policy_bind_date' in df_clean.columns:
    df_clean['policy_bind_date'] = pd.to_datetime(df_clean['policy_bind_date'], errors='coerce')
    df_clean['policy_bind_year'] = df_clean['policy_bind_date'].dt.year
    df_clean['policy_bind_month'] = df_clean['policy_bind_date'].dt.month
    df_clean.drop(columns=['policy_bind_date'], inplace=True)
else:
    print("Column 'policy_bind_date' does not exist.")


- Converted policy_bind_date to proper datetime format

- Extracted useful features: year, month

- Droping the original date column

## 7: Encode Categorical Variables

### 1: Identify Categorical Columns

In [14]:
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()


### 2: Label Encoding for Binary Categorical Variables

In [15]:
binary_cats = [col for col in categorical_cols if df_clean[col].nunique() == 2]
label_encoders = {}
for col in binary_cats:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col].astype(str))
    label_encoders[col] = le


I:

- Identified columns with only 2 unique values (binary)

- Used LabelEncoder to convert them to 0/1 format

### 3: One-Hot Encoding for Nominal Variables

In [16]:
nominal_cats = [col for col in categorical_cols if col not in binary_cats]
df_clean = pd.get_dummies(df_clean, columns=nominal_cats, drop_first=True)


I applied One-Hot Encoding to nominal categorical variables:

- Converting categories into binary columns (0/1)

- drop_first=True avoids multicollinearity by dropping the first category

## 8: Feature Scaling for Numeric Columns

In [17]:
numerical_cols = df_clean.select_dtypes(include=['int64', 'float64']).drop(columns=['fraud_reported']).columns.tolist()
scaler = StandardScaler()
df_clean[numerical_cols] = scaler.fit_transform(df_clean[numerical_cols])


- Identifing numerical columns

- Appling StandardScaler to normalize values (mean = 0, std = 1)

## 9: Preview Cleaned Data

In [18]:
print("Shape of cleaned data:", df_clean.shape)
df_clean.head()


Shape of cleaned data: (1000, 1145)


,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,capital-gains,capital-loss,...,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6
0,1.078140,0.990836,-0.095953,-0.222383,0.616705,-0.479476,-0.489529,1.076953,1.011331,0.953851,...,False,False,False,False,False,False,False,False,False,False
1,0.208995,0.334073,-0.791527,1.412784,-0.242521,1.697980,-0.461008,1.076953,-0.901927,0.953851,...,False,False,False,False,False,False,False,False,False,False
2,-0.608002,-1.088913,0.550566,1.412784,0.642233,1.697980,-0.984885,-0.928546,0.358023,0.953851,...,False,True,False,False,False,False,False,False,False,False
3,0.452355,0.224613,-1.239334,1.412784,0.652886,2.133471,1.491682,-0.928546,0.853388,-1.267577,...,False,False,False,False,False,True,False,False,False,False
4,0.208995,0.552994,-0.695834,-0.222383,1.341980,2.133471,1.527808,1.076953,1.467210,-0.683741,...,False,False,True,False,False,False,False,False,False,False


This gives an overview of:

- The new shape (rows × columns)

- A few sample rows after transformation

----

# 2: Data Understanding & Exploration

## 1: Inspecting the dataset

### 1: Dataset Overview
- Rows: 1,000

- Columns: 1,146
(Most of these are due to one-hot encoding of categorical variables.)


### 2: Data Types Summary
| Data Type        | Count                                 |
| ---------------- | ------------------------------------- |
| uint8          | 1,121 (from one-hot encoded features) |
| float64        | 20                                    |
| int64         | 4                                     |
| datetime64[ns] | 1                                     |


### 3: Target Variable: fraud_reported

This is the binary classification target:

| Value | Meaning        | Count | Percentage |
| ----- | -------------- | ----- | ---------- |
| `0`   | Not Fraudulent | 753   | 75.3%      |
| `1`   | Fraudulent     | 247   | 24.7%      |

Class imbalance is present — only ~25% of claims are fraudulent.

### 4: Sample of the Data

Snapshot of the cleaned and encoded dataset using:

In [19]:
df_clean.head()


,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,capital-gains,capital-loss,...,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6
0,1.078140,0.990836,-0.095953,-0.222383,0.616705,-0.479476,-0.489529,1.076953,1.011331,0.953851,...,False,False,False,False,False,False,False,False,False,False
1,0.208995,0.334073,-0.791527,1.412784,-0.242521,1.697980,-0.461008,1.076953,-0.901927,0.953851,...,False,False,False,False,False,False,False,False,False,False
2,-0.608002,-1.088913,0.550566,1.412784,0.642233,1.697980,-0.984885,-0.928546,0.358023,0.953851,...,False,True,False,False,False,False,False,False,False,False
3,0.452355,0.224613,-1.239334,1.412784,0.652886,2.133471,1.491682,-0.928546,0.853388,-1.267577,...,False,False,False,False,False,True,False,False,False,False
4,0.208995,0.552994,-0.695834,-0.222383,1.341980,2.133471,1.527808,1.076953,1.467210,-0.683741,...,False,False,True,False,False,False,False,False,False,False


Getting a feel for how the data looks post-transformation (with all numeric values and encoded features).